## CAMADA GOLD — MODELAGEM ANALÍTICA

**Objetivo da Camada Gold**

Disponibilizar dados modelados em Esquema Estrela, prontos para responder às perguntas do MVP com consultas simples, rápidas e consistentes.

**Estrutura final**
- fato_despesa_ceap
- dim_parlamentar
- dim_tempo
- dim_tipo_despesa
- dim_partido
- dim_uf


In [0]:
%sql
USE CATALOG mvp_ed_ceap;
USE SCHEMA layer_gold;

In [0]:
%sql

--DIM_PARLAMENTAR
CREATE OR REPLACE TABLE mvp_ed_ceap.layer_gold.dim_parlamentar AS
SELECT DISTINCT
  id_parlamentar,
  nome_parlamentar,
  sigla_partido,
  sigla_uf_parlamentar AS sigla_uf,
  num_legislatura
FROM mvp_ed_ceap.layer_silver.silver_ceap_despesas
WHERE id_parlamentar IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql

--DIM_TEMPO
CREATE OR REPLACE TABLE mvp_ed_ceap.layer_gold.dim_tempo AS
SELECT DISTINCT
  CAST(date_format(data_emissao, 'yyyyMMdd') AS INT) AS id_tempo,
  data_emissao AS data,
  year(data_emissao) AS ano,
  month(data_emissao) AS mes,
  quarter(data_emissao) AS trimestre,
  date_format(data_emissao, 'MMMM') AS nome_mes
FROM mvp_ed_ceap.layer_silver.silver_ceap_despesas
WHERE data_emissao IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql

--DIM_TIPO_DESPESA
CREATE OR REPLACE TABLE mvp_ed_ceap.layer_gold.dim_tipo_despesa AS
SELECT DISTINCT
  id_tipo_despesa,
  descricao_tipo_despesa,
  id_especificacao_tipo,
  descricao_especificacao_tipo
FROM mvp_ed_ceap.layer_silver.silver_ceap_despesas
WHERE id_tipo_despesa IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql

--DIM_PARTIDO
CREATE OR REPLACE TABLE mvp_ed_ceap.layer_gold.dim_partido AS
SELECT DISTINCT
  sigla_partido AS id_partido,
  sigla_partido
FROM mvp_ed_ceap.layer_silver.silver_ceap_despesas
WHERE sigla_partido IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql

--DIM_UF
CREATE OR REPLACE TABLE mvp_ed_ceap.layer_gold.dim_uf AS
SELECT DISTINCT
  sigla_uf_parlamentar AS id_uf,
  sigla_uf_parlamentar AS sigla_uf
FROM mvp_ed_ceap.layer_silver.silver_ceap_despesas
WHERE sigla_uf_parlamentar IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql

--FATO_DESPESA_CEAP
CREATE OR REPLACE TABLE mvp_ed_ceap.layer_gold.fato_despesa_ceap AS
SELECT
  s.id_documento,
  s.id_parlamentar,
  CAST(date_format(s.data_emissao, 'yyyyMMdd') AS INT) AS id_tempo,
  s.id_tipo_despesa,
  s.sigla_partido AS id_partido,
  s.sigla_uf_parlamentar AS id_uf,
  s.valor_documento,
  s.valor_glosa,
  s.valor_liquido
FROM mvp_ed_ceap.layer_silver.silver_ceap_despesas s
WHERE s.valor_documento IS NOT NULL;


num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Contagem de registros
SELECT COUNT(*) FROM mvp_ed_ceap.layer_gold.fato_despesa_ceap;

-- Conferir joins
SELECT COUNT(*)
FROM mvp_ed_ceap.layer_gold.fato_despesa_ceap f
JOIN mvp_ed_ceap.layer_gold.dim_parlamentar p
  ON f.id_parlamentar = p.id_parlamentar;


COUNT(*)
231703
